In [1]:
# Import necessary libraries for the UI and our core logic
import sys
sys.path.append('..') # Add the parent directory (project root) to the Python path

from ipywidgets import Textarea, VBox, Button, Output, IntProgress, Layout
from IPython.display import display, clear_output, HTML
import time
import traceback

# Import our main function from the src package
from src.core import process_order_request

# --- Define the User Queries ---
user_orders = [
    "We had two burgers at 10 euros each and a large soda for 3 euros.",
    "My order was one salad for 12 euro and a bottle of water for 4 euro.",
    "The total for our food was 50 euros, and we had a bottle of wine for 25. The service was great, so add a 10 euro tip to the bill."
]

print("✅ System ready.")

✅ System ready.


In [2]:
def create_validator_ui():
    query_input = Textarea(
        value=user_orders[0],
        placeholder='Paste an order here...',
        description='Customer Order:',
        style={'description_width': 'initial'},
        layout={'width': '95%', 'height': '100px'}
    )
    run_button = Button(description='Generate DSL & Calculate', button_style='success', icon='play')
    progress_bar = IntProgress(value=0, min=0, max=10, description='Waiting...', bar_style='info', orientation='horizontal', layout={'visibility': 'hidden'})
    output_area = Output()

    def run_analysis(button):
        with output_area:
            clear_output(wait=True)
            run_button.disabled = True
            progress_bar.description = 'LLM processing...'
            progress_bar.layout.visibility = 'visible'
            progress_bar.value = 5

            # --- Call our core logic function ---
            result = process_order_request(query_input.value)
            
            progress_bar.value = 10

            if result["status"] == "success":
                display(HTML("<h4>1. LLM-Generated DSL Code</h4>"))
                display(HTML(f"<pre style='background-color:#f5f5f5; border: 1px solid #ccc; padding: 10px; border-radius: 5px;'>{result['llm_generated_dsl']}</pre>"))
                
                display(HTML("<h4>2. Interpreter Execution Result</h4>"))
                dsl_result = result['interpreter_result']
                result_html = f"""
                <table style="width:100%; border-collapse: collapse;">
                  <tr style="background-color:#f2f2f2;"><th style="text-align:left; padding:8px; border: 1px solid #ddd;">Component</th><th style="text-align:right; padding:8px; border: 1px solid #ddd;">Calculated Value</th></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Net Food Cost</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['net_food_cost']:.2f}</td></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Net Drink Cost</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['net_drink_cost']:.2f}</td></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Total Tax</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['total_tax']:.2f}</td></tr>
                  <tr style="font-weight:bold; background-color:#f2f2f2;"><td style="padding:8px; border: 1px solid #ddd;">FINAL BILL</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['final_bill']:.2f}</td></tr>
                </table>
                """
                display(HTML(result_html))
            else:
                print(f"🔴 PROCESSING ERROR: {result['message']}")

            run_button.disabled = False
            progress_bar.layout.visibility = 'hidden'
            progress_bar.value = 0

    run_button.on_click(run_analysis)
    print("--- DSL Validator ---")
    display(VBox([query_input, run_button, progress_bar, output_area]))

# --- Run the function to display the UI ---
create_validator_ui()

--- DSL Validator ---
